# Le but de ce notebook est d'identifier des caractéristiques très simples permettant de différencier des textes écrits par plusieurs auteurs
## (en l'occurrence Molière et Corneille)

In [2]:
import re
import os
import random
import math

# Liste des noms de fichiers contenant les œuvres de Molière.
# Pour cette preuve de concept (POC), j'ai tout regroupé dans un seul fichier texte (format ANSI):
#    Moliere.txt
# Les données proviennent de:
#    https://www.ebooksgratuits.org/ebooksfrance/moliere-oeuvres_completes_1.pdf
#    https://www.ebooksgratuits.org/ebooksfrance/moliere-oeuvres_completes_2.pdf

moliere_filenames = ["Moliere.txt"]

# Liste des noms de fichiers contenant les œuvres de Corneille.
# Pour ce POC, j'ai tout regroupé dans un seul fichier texte (format ANSI):
#    Corneille.txt
# Les données proviennent de:
#    https://www.ebooksgratuits.com/ebooksfrance/corneille_pierre_theatre_complet_i.pdf
#    https://www.ebooksgratuits.com/ebooksfrance/corneille_pierre_theatre_complet_ii.pdf
corneille_filenames = ["Corneille.txt"]

# le répertoire de travail
directory = os.path.abspath('')

# découpe une phrase en mots, en ne gardant que les lettres
# 'il fait beau !' => ['il', 'fait', 'beau']
def split_sentence(sentence):
    words = re.findall(r'\b[\w^\d]+\b', sentence.rstrip().lower())
    return words


# infique si la ligne 'ligne' est une ligne valide ou si elle doit être ignorée (par exemple si elle trop vide)
def ligne_valide(ligne: str) -> bool:
    mots = split_sentence(ligne)
    if len(mots)<= 2:
        return False # phrase trop petite
    if len(mots) >= 12:
        return False # phrase trop grande
    return True # phrase valide


# Pour tous les fichiers indiqués dans 'filenames' :
#   Pour toutes les lignes de chaque fichier :
#     Si la ligne est considérée comme valide (ni trop courte ni trop longue):
#       On l'ajoute au résultat que l'on va retourner
#     Sinon:
#       On ignore la ligne
def toutes_les_lignes_valides(filenames):
    result = set()
    for f in filenames:
        path = os.path.join(directory, f)
        with open(path) as file:
            for line in file:
                if ligne_valide(line):
                    result.add(line.rstrip())
    result = list(result)                    
    return result


# Retourne un dictionnaire mot => pourcentage, indiquant pour chaque mot le pourcentage de lignes contenant ce mot.
# 1.0 signifie que le mot est présent dans toutes les lignes, 0.0 signifie que le mot est totalement absent.
# Cela se rapproche du concept de Document Frequency (dans TF-IDF).
def calcul_pourcentage_phrases_contenant_mot(lignes) -> dict:
    nombre_de_phrases_contenant_mot = dict()
    for line in lignes:
        mots = set(split_sentence(line))
        for mot in mots:
            if mot in nombre_de_phrases_contenant_mot:
                nombre_de_phrases_contenant_mot[mot] += 1
            else:
                nombre_de_phrases_contenant_mot[mot] = 1
    result = dict()
    for mot, nombre_de_phrases in nombre_de_phrases_contenant_mot.items():
        result[mot] = nombre_de_phrases/len(lignes)
    return result


# divise le dataset d'entraînement en une partie train (90%) et une partie validation (10%)
def split_train_validation(lignes, percentage_in_train:float=0.9):
    random.shuffle(lignes)
    train_lignes = lignes[:int( percentage_in_train*len(lignes))]
    validation_lignes = lignes[len(train_lignes):]
    return train_lignes, validation_lignes
    

# fonction de score ultra simpliste:
#    1/ poour chaque mot de la phrase 'ligne' dont on veut déterminer l'auteur, 
#       on va regarder le % de phrase de chaque auteur contenant ce mot 
#       (dans le dictionaire 'pourcentage_phrases_contenant_mot')
#       Cela ressemble beaucoup à l'idée du Document Frequency du TF-IDF
#    2/ on fait la somme de tous ces %, et on retourne cette somme
#
# Par exemple, pour la phrase: "Il fait beau":
#    1/ on regarde le % de phrases de l'auteur contenant les mots 'Il' , 'fait', 'beau' (par exemple : 10%, 2%, 3%)
#    2/ on retourne la somme de ces pourcentages, ici 0.15 (15%)
def calcul_caracteristique(ligne: str, pourcentage_phrases_contenant_mot:dict):
    score = 0
    for mot in split_sentence(ligne):
        if mot in pourcentage_phrases_contenant_mot:
            score += pourcentage_phrases_contenant_mot[mot]
    return score


In [4]:
print(f'Working directory: {directory}')


# on extrait toutes les lignes valides des oeuvres de Molière    
lignes_moliere = toutes_les_lignes_valides(moliere_filenames)
random.shuffle(lignes_moliere)

# on extrait toutes les lignes valides des oeuvres de Corneille
lignes_corneille = toutes_les_lignes_valides(corneille_filenames)
random.shuffle(lignes_corneille)

# les 2 datasets (Molière & Corneille) doivent avoir la même taille
taille_dataset = min(len(lignes_moliere), len(lignes_corneille))
lignes_moliere = lignes_moliere[:taille_dataset]
lignes_corneille = lignes_corneille[:taille_dataset]

                
print(f'Lignes écrites par Molière: {len(lignes_moliere)}')
print(f'Lignes écrites par Corneille: {len(lignes_corneille)}')


# on divise les données d'entraînement en train et validation
train_lignes_moliere, validation_lignes_moliere = split_train_validation(lignes_moliere)
train_lignes_corneille, validation_lignes_corneille = split_train_validation(lignes_corneille)


# pour chaque mot de chaque ligne, on compte le % de lignes de chaque auteur contenant ce mot
pourcentage_phrases_moliere_contenant_mot = calcul_pourcentage_phrases_contenant_mot(train_lignes_moliere)
pourcentage_phrases_corneille_contenant_mot = calcul_pourcentage_phrases_contenant_mot(train_lignes_corneille)


# on mesure la performance de la caractéristique calculée plus haut à déterminer
# si une ligne a été écritre par Molière (TP), et on calcule la matrice de confusion associée

TP = 0 # y_true = Molière ,  y_pred = Molière
TN = 0 # y_true = Corneille, y_pred = Corneille
FN = 0 # y_true = Molière,   y_pred = Corneille
FP = 0 # y_true = Corneille, y_pred = Molière 


# Pour toutes les lignes (du jeu de données de validation) écrites par Molière,
# on vérifie si notre caractéristique est capable de les identifier correctement comme étant écrites par Molière (TP)
for l in validation_lignes_moliere:
    score_moliere = calcul_caracteristique(l, pourcentage_phrases_moliere_contenant_mot)
    score_corneille = calcul_caracteristique(l, pourcentage_phrases_corneille_contenant_mot)
    if score_moliere>score_corneille:
        if TP == 0:
            print('Exemple de TP (Texte de Molière, bien identifié):')
            print('\t\t'+l)
            print(f'\t\tScore Molière: {round(score_moliere,4)}, Score Corneille: {round(score_corneille,4)}')
        TP += 1
    else:
        if FN == 0:
            print('Exemple de FN (Texte de Molière, mal identifié):')
            print('\t\t'+l)
            print(f'\t\tScore Molière: {round(score_moliere,4)}, Score Corneille: {round(score_corneille,4)}')
        FN += 1
        
# Pour toutes les lignes (du jeu de données de validation) écrites par Corneille,
# on vérifie si notre caractéristique est capable de les identifier correctement comme n'étant pas écrites par Molière (TN)
for l in validation_lignes_corneille:
    score_moliere = calcul_caracteristique(l, pourcentage_phrases_moliere_contenant_mot)
    score_corneille = calcul_caracteristique(l, pourcentage_phrases_corneille_contenant_mot)
    if score_corneille>score_moliere:
        if TN == 0:
            print('Exemple de TN (Texte de Corneille, bien identifié):')
            print('\t\t'+l)
            print(f'\t\tScore Molière: {round(score_moliere,4)}, Score Corneille: {round(score_corneille,4)}')
        TN += 1
    else:
        if FP == 0:
            print('Exemple de FP (Texte de Corneille, mal identifié):')
            print('\t\t'+l)
            print(f'\t\tScore Molière: {round(score_moliere,4)}, Score Corneille: {round(score_corneille,4)}')
        FP += 1

print(f'TP: {TP}, TN: {TN}, FN: {FN}, FP: {FP}')
print(f'Accuracy: {(TP+TN)/(TP+TN+FP+FN)}')


    


Working directory: C:\Projects\ChallengeDataTest\Stylometrie
Lignes écrites par Molière: 25342
Lignes écrites par Corneille: 25342
Exemple de TP (Texte de Molière, bien identifié):
		Fi ! cela sent mauvais, et je suis tout gâté :
		Score Molière: 0.4276, Score Corneille: 0.4099
Exemple de FN (Texte de Molière, mal identifié):
		Qui fait dedans vos mains trouver cette peinture ?
		Score Molière: 0.1482, Score Corneille: 0.1491
Exemple de TN (Texte de Corneille, bien identifié):
		Il rallume en son coeur déjà quelque espérance,
		Score Molière: 0.2663, Score Corneille: 0.3299
Exemple de FP (Texte de Corneille, mal identifié):
		Puisque avant ce portrait on aura votre coeur ;
		Score Molière: 0.2027, Score Corneille: 0.1743
TP: 930, TN: 2042, FN: 1605, FP: 493
Accuracy: 0.5861932938856016
